In [183]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [184]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

base_url = 'https://www.glyndebourne.com/about-us/our-archive/performance-archive/PerformanceSearchForm?Opera=58&Year=&Name=&action_doPerformanceSearch=Search'

In [185]:
import re

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', 
        '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
        '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', 
        '31', '1', '2', '3', '4', '5', '6', '7', '8', '9']    

def get_dates(date_str, year):

    dates_list = []        
    dates = re.split('; |, |\. | |',date_str)    
    year = str(year)
    month = ''
    day = ''
    
    if any(ext in dates for ext in dow):
        for idx in range(len(dates)):
            if dates[idx] in days:
                day = dates[idx]
            if dates[idx] in months:
                month = dates[idx]
            if dates[idx] in years:
                year = dates[idx]
                new_date = year + '-' + month + '-' + day
                dates_list.append(new_date)
    else:            
        for idx in range(len(dates)):
            if dates[idx] in years:
                year = dates[idx]
            if dates[idx] in months:
                month = dates[idx]
            if dates[idx] in days:
                new_date = year + '-' + month + '-' + str(dates[idx])
                dates_list.append(new_date)
    return dates_list

In [186]:
listings = []

r  = requests.get(base_url, headers=headers)
soup = BeautifulSoup(r.text, "html5lib")

decades = soup.find_all(attrs={'class': 'large-4 medium-6 small-12 columns card'})
last_decade = soup.find_all(attrs={'class': 'large-4 medium-6 small-12 columns medium-centered large-centered card'})
decades.append(last_decade[0])

for decade in decades:
    seasons = decade.find_all('a')[1:-1]
    for season in seasons:    
        url = 'https://www.lyricopera.org' + season.get('href')  
        print(url)
        year = season.get('href')
        year = year.split('/')[-1].split('-')[0]
        r  = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, "html5lib")
        
        if int(year) <= 2007:        
            results = soup.find_all('strong')
            for result in results:
                try:
                    work = result.find('em').get_text() 
                    composer = result.find('em').next_sibling.next_sibling.strip()
                    dates = result.parent.find_all('em')[1].get_text().split(' / ')[0]
                    dates = dates.replace(' (matinee)', '')
                    dates = get_dates(dates, year)
                    for date in dates:
                        date = datetime.strptime(date, '%Y-%b-%d').date()            

                        listing = []
                        listing.append(work)                        
                        listing.append(composer)
                        listing.append(date)
                        listings.append(listing)
                except:
                    None
        else:
            results = soup.find_all('h3')
            
            for result in results:
                work = result.get_text().strip()
                if result.next_sibling.next_sibling.get_text().strip()!='':
                    data = result.next_sibling.next_sibling.get_text(strip=True, separator="---")
                    if len(data.split("---"))>1:
                        composer = data.split("---")[0]
                        dates = data.split("---")[1].split('/')[0]
                    else:
                        composer = data
                        dates = result.next_sibling.next_sibling.next_sibling            
                else:
                    composer = result.next_sibling.next_sibling.get_text()
                    dates = result.next_sibling.next_sibling.next_sibling.split('/')[0]

                dates = get_dates(dates, year)
                for date in dates:
                    try:
                        date = datetime.strptime(date, '%Y-%b-%d').date()
                    except:
                        print(date)
                    listing = []
                    listing.append(work)                        
                    listing.append(composer)
                    listing.append(date)
                    print(listing)
                    listings.append(listing)

#Saving results into dataframe and folder            
import pandas as pd                     
headers = ['work', 'composer', 'date']
df = pd.DataFrame(listings, columns=headers)
df.to_csv('../../data/processed/listings/chicago_lyric.csv')                            

https://www.lyricopera.org/about/production-archives/1954-1959-performance-and-cast-archive/1954-performance-and-cast-archive


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


https://www.lyricopera.org/about/production-archives/1954-1959-performance-and-cast-archive/1955-performance-and-cast-archive
https://www.lyricopera.org/about/production-archives/1954-1959-performance-and-cast-archive/1956-performance-and-cast-archive
https://www.lyricopera.org/about/production-archives/1954-1959-performance-and-cast-archive/1957-performance-and-cast-archive
https://www.lyricopera.org/about/production-archives/1954-1959-performance-and-cast-archive/1958-performance-and-cast-archive
https://www.lyricopera.org/about/production-archives/1960-1969-performance-and-cast-archive/1960-performance-and-cast-archive
https://www.lyricopera.org/about/production-archives/1960-1969-performance-and-cast-archive/1961-performance-and-cast-archive
https://www.lyricopera.org/about/production-archives/1960-1969-performance-and-cast-archive/1962-performance-and-cast-archive
https://www.lyricopera.org/about/production-archives/1960-1969-performance-and-cast-archive/1963-performance-and-cast-

['Macbeth', 'Giuseppe Verdi', datetime.date(2010, 10, 1)]
['Macbeth', 'Giuseppe Verdi', datetime.date(2010, 10, 6)]
['Macbeth', 'Giuseppe Verdi', datetime.date(2010, 10, 9)]
['Macbeth', 'Giuseppe Verdi', datetime.date(2010, 10, 12)]
['Macbeth', 'Giuseppe Verdi', datetime.date(2010, 10, 15)]
['Macbeth', 'Giuseppe Verdi', datetime.date(2010, 10, 18)]
['Macbeth', 'Giuseppe Verdi', datetime.date(2010, 10, 21)]
['Macbeth', 'Giuseppe Verdi', datetime.date(2010, 10, 24)]
['Macbeth', 'Giuseppe Verdi', datetime.date(2010, 10, 30)]
['Carmen', 'Georges Bizet', datetime.date(2010, 10, 13)]
['Carmen', 'Georges Bizet', datetime.date(2010, 10, 16)]
['Carmen', 'Georges Bizet', datetime.date(2010, 10, 20)]
['Carmen', 'Georges Bizet', datetime.date(2010, 10, 22)]
['Carmen', 'Georges Bizet', datetime.date(2010, 10, 25)]
['Carmen', 'Georges Bizet', datetime.date(2010, 10, 29)]
['Carmen', 'Georges Bizet', datetime.date(2010, 3, 12)]
['Carmen', 'Georges Bizet', datetime.date(2010, 3, 15)]
['Carmen', 'George

https://www.lyricopera.org/about/production-archives/2010-2014-performance-and-cast-archive/2014-15-performance-and-cast-archive
https://www.lyricopera.org/about/production-archives/2010-2014-performance-and-cast-archive/2015-16-performance-and-cast-archive
